In [10]:
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.api as sm
import warnings
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from pandas import Series
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import StandardScaler
from math import sqrt
from datetime import datetime,timedelta
from dateutil.rrule import rrule, HOURLY
from statsmodels.tsa.arima.model import ARIMA
import itertools
import math
import time

from keras.models import Model, Sequential
from keras.layers import GRU, Dense, LSTM, Dropout, Bidirectional, SimpleRNN
from tensorflow.keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics

warnings.filterwarnings("ignore")

def percentage_error(actual, predicted):
    res = np.empty(actual.shape)
    for j in range(actual.shape[0]):
        if actual[j] != 0:
            res[j] = (actual[j] - predicted[j]) / actual[j]
        else:
            res[j] = predicted[j] / np.mean(actual)
    return res    
def percentage_error(actual, predicted):
    res = np.empty(actual.shape)
    for j in range(actual.shape[0]):
        if actual[j] != 0:
            res[j] = (actual[j] - predicted[j]) / actual[j]
        else:
            res[j] = predicted[j] / np.mean(actual)
    return res    

def mae(actual, predicted):
    res = np.empty(actual.shape)
    r = 0
    n = 0
    for j in range(actual.shape[0]):
            r += abs(actual[j] - predicted[j])
            n+=1
    return r/n    

def mse(actual, predicted):
    res = np.empty(actual.shape)
    r = 0
    n=0
    for j in range(actual.shape[0]):
            r += (actual[j] - predicted[j])**2
            n+=1
    return r/n    

def rmse (actual, predicted):
    r = mse(actual,predicted)
    return math.sqrt(r)

totalmae=0
totalmse=0
totalrmse=0
totalduration=0

frames=[]
totalmape=0
no_predictions = 24
lag = 24
batch_size=24
epochs = 100

#ids = [28079024, 28079038, 28079008, 28079040, 28079036, 28079018, 28079011, 28079004, 28079016, 28079039, 28079027]
#ids = [28079024, 28079038, 28079008, 28079047, 28079050, 28079048, 28079099, 28079026, 28079006, 28079022, 28079001, 28079015]
ids = [28079008, 28079047, 28079026, 28079006, 28079022, 28079015]
for k in ids:
    start_exec = time.time()
    dffinal=pd.read_csv("processed-"+str(k)+".csv", index_col='date',parse_dates=True)
    #dffinal=pd.read_csv("cleaned.csv", index_col='date',parse_dates=True)

    dffinal=dffinal.drop(columns=['NO','CH4','BEN','CO','EBE','MXY','NMHC','O_3','NOx','OXY','PM10','PXY','SO_2','TCH','TOL','station'])
    dffinal.dropna()
    dffinal.drop(dffinal.index[365*24*2:],inplace=True)
    


    sc_in = MinMaxScaler(feature_range=(0, 1))
    dffinal['PM25'] = sc_in.fit_transform(dffinal[['PM25']])
    sc_in_no2 = MinMaxScaler(feature_range=(0, 1))
    dffinal['NO_2'] = sc_in_no2.fit_transform(dffinal[['NO_2']])
    
    #split data
    train_size=  int(len(dffinal) *0.6)
    valid_size = (int)((int(len(dffinal)) - train_size)/2)
    test_size = (int(len(dffinal)) - train_size-valid_size)

    train = dffinal[:train_size].dropna()
    valid = dffinal[train_size:train_size+valid_size].dropna()
    test = dffinal[train_size+valid_size:].dropna()

    
    train.rename(columns={'PM25':'y','NO_2':'NO_2'}, inplace=True)
    train.index.names = ['ds']
    train.reset_index(inplace=True)
    #print(train)
      
    model = Prophet(**{'changepoint_prior_scale':0.1, 'seasonality_prior_scale':0.01, 'growth':'linear'})
    #model.add_seasonality(name='daily', period=24, fourier_order=12, prior_scale=0.1)
    #model.add_seasonality(name='weekly', period=24*7, fourier_order=3*24, prior_scale=0.1)

    model.fit(train)
    future = model.make_future_dataframe(periods=no_predictions, freq='H')
    future.tail()
    
    
    #print('Future:',future.tail(no_predictions))
    #print('Valid:',valid.head(no_predictions))
    
    forecast = model.predict(future)
    #print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(no_predictions))    
    evaluate = pd.DataFrame(columns=['timestamp','prediction','PM25'])
    for i in range(len(forecast)-no_predictions,len(forecast)):
        row = [forecast.iloc[i].ds,forecast.iloc[i].yhat,valid.iloc[i-len(forecast)+no_predictions].PM25]
        a_series = pd.Series(row, index = evaluate.columns)
        evaluate = evaluate.append(a_series, ignore_index=True)
#        print(forecast.iloc[i].ds,'-',forecast.iloc[i].yhat,' ',valid.iloc[i-len(forecast)+24].PM25)
     
        
    
    print(evaluate)
    evaluate['mape']=abs((evaluate['PM25']-evaluate['prediction'])/evaluate['PM25'])
    vmape = np.mean(np.abs(percentage_error(np.asarray(evaluate['PM25']), np.asarray(evaluate['prediction'])))) * 100
    vmae = mae(np.asarray(evaluate['PM25']), np.asarray(evaluate['prediction']))
    vmse = mse(np.asarray(evaluate['PM25']), np.asarray(evaluate['prediction']))
    vrmse = rmse(np.asarray(evaluate['PM25']), np.asarray(evaluate['prediction']))
    end_exec = time.time()
    duration = end_exec-start_exec
    print("MAPE",k,'-',vmape);
    print("MAE",k,'-',vmae);
    print("MSE",k,'-',vmse);
    print("RMSE",k,'-',vrmse);
    print("duration",k,duration)
    totalmape += vmape
    totalmae += vmae
    totalmse += vmse
    totalrmse += vrmse
    totalduration += duration
    
    #fig, ax = plt.subplots(1, 1)
    #forecasts.plot(title="Forecasts", ax=ax)
#    forecasts.plot(kind='kde', title='Forecast', ax=ax[1])
#    plt.show();

#    predictions = model.predict(start=train_size, end=train_size+no_predictions-1)
#    print("Type predictions:",type(predictions))
#    pred = pd.DataFrame(predictions)
#    pred.index = list(pred.index)
#    print("Index pred:",pred.index)
#    print("Index test:",test_X.index)
#    a = pred.loc[train_size]
#    b = test_X.loc[train_size]
#    print("A:",a)
#    print("B:",b)
#    print("Type pred:",type(pred))
#    print("Predictions:")
#    print(pred);
#    tape = 0
#    for i in range(train_size, train_size+no_predictions-1):
#        ape = abs(pred.loc[i].predicted_mean-test_X.loc[i].PM25)/abs(test_X.loc[i].PM25)*100
#        tape = tape + ape
#    mape = tape/no_predictions;
#    print("MAPE:",mape)
#    print("Accuracy:",100.0-mape)
#    totalmape = totalmape + mape
overallmape = totalmape/len(ids)
overallmae = totalmae/len(ids)
overallmse = totalmse/len(ids)
overallrmse = totalrmse/len(ids)
overallduration = totalduration/len(ids)
print("Overall MAPE:",overallmape)
print("Overall accuracy:",100.0-overallmape)
print("Overall MAE:",overallmae)
print("Overall MSE:",overallmse)
print("Overall RMSE:",overallrmse)
print("Overall duration:",overallduration)

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


              timestamp prediction      PM25
0   2013-01-04 10:00:00    0.17121     0.125
1   2013-01-04 11:00:00   0.165524     0.125
2   2013-01-04 12:00:00   0.149215  0.114583
3   2013-01-04 13:00:00   0.132112  0.135417
4   2013-01-04 14:00:00   0.121502  0.208333
5   2013-01-04 15:00:00   0.117862    0.1875
6   2013-01-04 16:00:00   0.117239  0.177083
7   2013-01-04 17:00:00   0.117253  0.166667
8   2013-01-04 18:00:00   0.120371  0.229167
9   2013-01-04 19:00:00   0.131124  0.260417
10  2013-01-04 20:00:00   0.150141      0.25
11  2013-01-04 21:00:00   0.171181  0.302083
12  2013-01-04 22:00:00   0.184457  0.333333
13  2013-01-04 23:00:00   0.183487  0.354167
14  2013-01-05 00:00:00   0.169323    0.3125
15  2013-01-05 01:00:00   0.148685  0.395833
16  2013-01-05 02:00:00   0.128342  0.479167
17  2013-01-05 03:00:00   0.111312  0.427083
18  2013-01-05 04:00:00   0.097965    0.1875
19  2013-01-05 05:00:00   0.089724  0.166667
20  2013-01-05 06:00:00   0.090353  0.145833
21  2013-0

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


              timestamp prediction      PM25
0   2011-03-04 01:00:00   0.181423  0.141672
1   2011-03-04 02:00:00   0.166742  0.151791
2   2011-03-04 03:00:00    0.15442  0.131552
3   2011-03-04 04:00:00   0.144236  0.121433
4   2011-03-04 05:00:00   0.136835  0.121433
5   2011-03-04 06:00:00   0.136287  0.131552
6   2011-03-04 07:00:00   0.147363  0.121433
7   2011-03-04 08:00:00    0.16955  0.091075
8   2011-03-04 09:00:00   0.194031  0.111313
9   2011-03-04 10:00:00   0.207864  0.151791
10  2011-03-04 11:00:00    0.20288  0.121433
11  2011-03-04 12:00:00   0.181867  0.141672
12  2011-03-04 13:00:00   0.156506  0.121433
13  2011-03-04 14:00:00   0.138687  0.131552
14  2011-03-04 15:00:00   0.132688  0.121433
15  2011-03-04 16:00:00   0.134564  0.111313
16  2011-03-04 17:00:00    0.13831  0.111313
17  2011-03-04 18:00:00   0.142263  0.111313
18  2011-03-04 19:00:00   0.149673  0.131552
19  2011-03-04 20:00:00   0.163507  0.151791
20  2011-03-04 21:00:00   0.181286  0.141672
21  2011-0

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


              timestamp prediction      PM25
0   2005-04-23 13:00:00   0.182911  0.126258
1   2005-04-23 14:00:00    0.16892  0.165153
2   2005-04-23 15:00:00   0.159793  0.119018
3   2005-04-23 16:00:00   0.155248  0.111411
4   2005-04-23 17:00:00   0.155564  0.130307
5   2005-04-23 18:00:00   0.161814  0.110429
6   2005-04-23 19:00:00   0.173722  0.244294
7   2005-04-23 20:00:00   0.187836  0.208834
8   2005-04-23 21:00:00   0.198532  0.206074
9   2005-04-23 22:00:00   0.201422  0.189264
10  2005-04-23 23:00:00   0.196108  0.188282
11  2005-04-24 00:00:00   0.185737  0.175951
12  2005-04-24 01:00:00   0.173909  0.175706
13  2005-04-24 02:00:00   0.162143  0.221043
14  2005-04-24 03:00:00   0.150394  0.184724
15  2005-04-24 04:00:00   0.139839  0.168098
16  2005-04-24 05:00:00   0.134521  0.162147
17  2005-04-24 06:00:00    0.13937  0.141963
18  2005-04-24 07:00:00   0.155767  0.123926
19  2005-04-24 08:00:00   0.178702  0.158589
20  2005-04-24 09:00:00   0.198628  0.158037
21  2005-0

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


              timestamp prediction      PM25
0   2005-04-25 13:00:00   0.118344  0.110985
1   2005-04-25 14:00:00   0.107841  0.105711
2   2005-04-25 15:00:00   0.101433  0.105754
3   2005-04-25 16:00:00   0.099434  0.088666
4   2005-04-25 17:00:00    0.10269  0.114342
5   2005-04-25 18:00:00   0.112444  0.072973
6   2005-04-25 19:00:00    0.12807  0.108152
7   2005-04-25 20:00:00   0.145426  0.074542
8   2005-04-25 21:00:00   0.158241  0.087228
9   2005-04-25 22:00:00   0.161861   0.09721
10  2005-04-25 23:00:00   0.156094  0.114952
11  2005-04-26 00:00:00   0.144575  0.143723
12  2005-04-26 01:00:00   0.131362  0.140715
13  2005-04-26 02:00:00   0.118169  0.177986
14  2005-04-26 03:00:00   0.104888  0.106103
15  2005-04-26 04:00:00   0.092567   0.09211
16  2005-04-26 05:00:00   0.085301  0.082781
17  2005-04-26 06:00:00   0.088375   0.08435
18  2005-04-26 07:00:00   0.103744  0.080994
19  2005-04-26 08:00:00   0.126952  0.125022
20  2005-04-26 09:00:00   0.148706  0.157062
21  2005-0

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


              timestamp prediction      PM25
0   2005-04-24 13:00:00   0.107095  0.060614
1   2005-04-24 14:00:00   0.081122  0.060614
2   2005-04-24 15:00:00   0.064996  0.062421
3   2005-04-24 16:00:00   0.054232  0.055917
4   2005-04-24 17:00:00   0.046613  0.045709
5   2005-04-24 18:00:00    0.04653  0.113279
6   2005-04-24 19:00:00   0.060028  0.078139
7   2005-04-24 20:00:00   0.086185  0.085005
8   2005-04-24 21:00:00   0.114426  0.079855
9   2005-04-24 22:00:00   0.131329   0.08383
10  2005-04-24 23:00:00   0.130737  0.065854
11  2005-04-25 00:00:00   0.117479  0.063234
12  2005-04-25 01:00:00   0.101379  0.062331
13  2005-04-25 02:00:00   0.087979  0.087444
14  2005-04-25 03:00:00   0.075494  0.063957
15  2005-04-25 04:00:00   0.060843  0.048058
16  2005-04-25 05:00:00    0.04779  0.050136
17  2005-04-25 06:00:00   0.047553  0.048961
18  2005-04-25 07:00:00   0.069631  0.050587
19  2005-04-25 08:00:00    0.11095  0.095303
20  2005-04-25 09:00:00   0.154268  0.134417
21  2005-0